# Tutorial 1: Producing a Training Dataset

This tutorial guides you through the complete process of generating a training dataset from raw manufacturing data.

## Overview

The dataset generation involves two main pipelines:
1. **Target Pipeline (IST)**: Processes resistance test data to create `df_trg.csv`
2. **Input Pipeline (Process)**: Processes manufacturing parameters and combines with target data

## Prerequisites

Before starting, ensure you have:
- Installed all dependencies (`pip install -r requirements.txt`)
- Obtained access to the raw data files
- Placed data files according to `data/README_DATA.md`

## Step 0: Setup

First, let's set up the environment and verify the data structure.

In [ ]:
import sys
import os
from os.path import exists, join, dirname, abspath

# Add project root to path
PROJECT_ROOT = dirname(abspath(os.getcwd()))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print(f"Project root: {PROJECT_ROOT}")

In [ ]:
# Verify data structure
from proT_pipeline.labels import get_target_dirs, get_input_dirs, get_root_dir

ROOT = get_root_dir()
TARGET_INPUT, TARGET_BUILDS = get_target_dirs(ROOT)

print("Checking data directories...")
print(f"  Target input dir: {TARGET_INPUT}")
print(f"    Exists: {exists(TARGET_INPUT)}")
print(f"  Target builds dir: {TARGET_BUILDS}")
print(f"    Exists: {exists(TARGET_BUILDS)}")

## Step 1: Process Target Data (IST)

The target pipeline processes the raw IST (Insulation Stress Test) resistance data.

### Configuration Parameters

| Parameter | Description | Typical Value |
|-----------|-------------|---------------|
| `build_id` | Output folder name | e.g., "my_build_2026" |
| `grouping_method` | How to group samples | "panel" or "column" |
| `max_len` | Maximum sequence length | 200 |
| `filter_type` | Coupon type filter | "C" (canary) or "P" (product) |
| `max_len_mode` | How to handle long sequences | "clip" or "remove" |
| `mean_bool` | Calculate mean across groups | False |
| `std_bool` | Calculate std across groups | False |

In [ ]:
# Configuration for target pipeline
BUILD_ID = "tutorial_build"          # Change this to your build name
GROUPING_METHOD = "panel"             # Group by panel (individual samples)
MAX_LEN = 200                         # Maximum 200 thermal cycles
FILTER_TYPE = "C"                     # Canary coupons only
UNI_METHOD = "clip"                   # Clip for uniform length
MAX_LEN_MODE = "clip"                 # Clip sequences exceeding max_len
MEAN_BOOL = False                     # Don't calculate mean (keep individual measurements)
STD_BOOL = False                      # Don't calculate std

In [ ]:
# Run target pipeline
from proT_pipeline.target_processing.main import main as target_main

print("Running target (IST) pipeline...")
print(f"  Build ID: {BUILD_ID}")
print(f"  Grouping: {GROUPING_METHOD}")
print(f"  Max length: {MAX_LEN}")
print()

target_main(
    build_id=BUILD_ID,
    grouping_method=GROUPING_METHOD,
    grouping_column=None,
    max_len=MAX_LEN,
    filter_type=FILTER_TYPE,
    uni_method=UNI_METHOD,
    max_len_mode=MAX_LEN_MODE,
    mean_bool=MEAN_BOOL,
    std_bool=STD_BOOL
)

print("\nTarget pipeline complete!")
print(f"Output: data/target/builds/{BUILD_ID}/df_trg.csv")

In [ ]:
# Verify output and preview
import pandas as pd

df_trg_path = join(TARGET_BUILDS, BUILD_ID, "df_trg.csv")
print(f"Loading: {df_trg_path}")

df_trg = pd.read_csv(df_trg_path)
print(f"\nTarget dataframe shape: {df_trg.shape}")
print(f"Unique groups (samples): {df_trg['group'].nunique()}")
print(f"Unique variables: {df_trg['variable'].unique()}")
print(f"\nFirst few rows:")
df_trg.head(10)

## Step 2: Copy Target to Build Folder

The input pipeline expects `df_trg.csv` in the build's control folder. We need to copy it there.

In [ ]:
# Configuration for input pipeline
DATASET_ID = "tutorial_build"         # Must match or create this folder in data/builds/

# Create build folder structure if it doesn't exist
BUILD_DIR = join(ROOT, "data", "builds", DATASET_ID)
CONTROL_DIR = join(BUILD_DIR, "control")
OUTPUT_DIR = join(BUILD_DIR, "output")

os.makedirs(CONTROL_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Build directory: {BUILD_DIR}")
print(f"Control directory: {CONTROL_DIR}")
print(f"Output directory: {OUTPUT_DIR}")

In [ ]:
# Copy df_trg.csv to control folder
import shutil

source_path = join(TARGET_BUILDS, BUILD_ID, "df_trg.csv")
dest_path = join(CONTROL_DIR, "df_trg.csv")

if exists(source_path):
    shutil.copy(source_path, dest_path)
    print(f"Copied df_trg.csv to {dest_path}")
else:
    print(f"ERROR: Source file not found: {source_path}")
    print("Make sure the target pipeline ran successfully.")

## Step 3: Prepare Control Files

The input pipeline requires several control files. You should copy these from an existing build or create them according to the schema in `data/builds/template_build/control/README.md`.

**Required files:**
- `config.yaml` - Points to input dataset folder
- `lookup_selected.xlsx` - Variable selection per process
- `steps_selected.xlsx` - Process steps to include
- `Prozessfolgen_MSEI.xlsx` - Layer/occurrence mapping

In [ ]:
# Check for required control files
required_files = [
    "config.yaml",
    "df_trg.csv",
    "lookup_selected.xlsx",
    "steps_selected.xlsx",
    "Prozessfolgen_MSEI.xlsx"
]

print("Checking control files:")
all_present = True
for f in required_files:
    path = join(CONTROL_DIR, f)
    status = "✓" if exists(path) else "✗ MISSING"
    if not exists(path):
        all_present = False
    print(f"  {status} {f}")

if not all_present:
    print("\n⚠️ Some control files are missing!")
    print("Copy them from an existing build or create them following the schema.")

## Step 4: Run Input Pipeline

Now we can run the input pipeline to process manufacturing data and generate the final dataset.

### Configuration Parameters

| Parameter | Description | Typical Value |
|-----------|-------------|---------------|
| `dataset_id` | Build folder name | Same as your folder in data/builds/ |
| `missing_threshold` | Max % missing values per variable | 30 |
| `use_stratified_split` | Enable stratified train/test split | True |
| `train_ratio` | Proportion for training set | 0.8 |
| `n_bins` | Number of bins for stratification | 50 |
| `grouping_method` | How samples are grouped | "panel" |

In [ ]:
# Input pipeline configuration
MISSING_THRESHOLD = 30                # Remove variables with >30% missing values
USE_STRATIFIED_SPLIT = True           # Enable stratified splitting
STRATIFIED_METRIC = 'rarity_last_value'  # Metric for stratification
TRAIN_RATIO = 0.8                     # 80% train, 20% test
N_BINS = 50                           # Bins for stratification
SPLIT_SHUFFLE = False                 # Don't shuffle within bins
SPLIT_SEED = 42                       # Random seed for reproducibility
DEBUG = False                         # Set True for quick test with subset

In [ ]:
# Run input pipeline
from proT_pipeline.main import main as input_main

print("Running input (process) pipeline...")
print(f"  Dataset ID: {DATASET_ID}")
print(f"  Missing threshold: {MISSING_THRESHOLD}%")
print(f"  Stratified split: {USE_STRATIFIED_SPLIT}")
print(f"  Train ratio: {TRAIN_RATIO}")
print()

input_main(
    dataset_id=DATASET_ID,
    missing_threshold=MISSING_THRESHOLD,
    select_test=False,
    use_stratified_split=USE_STRATIFIED_SPLIT,
    stratified_metric=STRATIFIED_METRIC,
    train_ratio=TRAIN_RATIO,
    n_bins=N_BINS,
    split_shuffle=SPLIT_SHUFFLE,
    split_seed=SPLIT_SEED,
    grouping_method=GROUPING_METHOD,
    grouping_column=None,
    debug=DEBUG
)

print("\nInput pipeline complete!")
print(f"Output: data/builds/{DATASET_ID}/output/")

## Step 5: Verify Output

Let's examine the generated dataset to ensure everything worked correctly.

In [ ]:
import numpy as np

# Load the generated dataset
dataset_path = join(OUTPUT_DIR, f"ds_{DATASET_ID}", "data.npz")

if exists(dataset_path):
    data = np.load(dataset_path)
    X = data['x']
    Y = data['y']
    
    print("Dataset loaded successfully!")
    print(f"\nX (input) shape: {X.shape}")
    print(f"  - Samples: {X.shape[0]}")
    print(f"  - Max sequence length: {X.shape[1]}")
    print(f"  - Features: {X.shape[2]}")
    print(f"\nY (target) shape: {Y.shape}")
    print(f"  - Samples: {Y.shape[0]}")
    print(f"  - Max sequence length: {Y.shape[1]}")
    print(f"  - Features: {Y.shape[2]}")
else:
    print(f"Dataset not found at: {dataset_path}")

In [ ]:
# Check vocabulary files
import json

vocab_files = [
    "group_vocabulary.json",
    "process_vocabulary.json",
    "variables_vocabulary.json_input",
    "variables_vocabulary.json_trg",
    "features_dict"
]

print("Vocabulary files:")
for vf in vocab_files:
    path = join(OUTPUT_DIR, vf)
    if exists(path):
        with open(path, 'r') as f:
            vocab = json.load(f)
        print(f"\n{vf}: {len(vocab)} entries")
        if len(vocab) <= 10:
            print(f"  {vocab}")
        else:
            print(f"  First 5: {dict(list(vocab.items())[:5])}")

In [ ]:
# Check train/test splits
train_path = join(OUTPUT_DIR, f"ds_{DATASET_ID}", "train_data.npz")
test_path = join(OUTPUT_DIR, f"ds_{DATASET_ID}", "test_data.npz")

if exists(train_path) and exists(test_path):
    train_data = np.load(train_path)
    test_data = np.load(test_path)
    
    print("Train/Test splits:")
    print(f"  Train samples: {train_data['x'].shape[0]}")
    print(f"  Test samples: {test_data['x'].shape[0]}")
    print(f"  Train ratio: {train_data['x'].shape[0] / (train_data['x'].shape[0] + test_data['x'].shape[0]):.2%}")
else:
    print("Train/test splits not found.")
    print("Run with use_stratified_split=True to generate splits.")

## Summary

You have successfully generated a training dataset! The output includes:

| File | Description |
|------|-------------|
| `data.npz` | Full dataset (X and Y arrays) |
| `train_data.npz` | Training split |
| `test_data.npz` | Test split |
| `*_vocabulary.json` | Mapping dictionaries |
| `features_dict` | Feature index documentation |
| `sample_metrics.parquet` | Sample-level metrics for analysis |

## Next Steps

1. Use the dataset for training your transformer model
2. See Tutorial 2 for generating prediction datasets (without targets)
3. Refer to `INTEGRATION_GUIDE.md` for advanced configuration options